In [1]:
import requests,json
import pandas as pd
from requests_html import HTML
import re

,ID,title,content,createdAt,commentCount,forumName,forumName,gender,likeCount,reactions,topics
0,234001756,談談憂鬱症,誰來跟我分享分享有關憂鬱症\n看到低卡幾篇文章\n又想到吳某某之前新聞的評論\n憂鬱症患者發...,2020-07-05T16:30:52.129Z,5,心情,心情,F,2,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[心情, 憂鬱症]"


In [2]:
def Crawl(ID):
    link = 'https://www.dcard.tw/_api/posts/' + str(ID)
    #偽裝成瀏覽器，(因為Dcard Server有用cloudflare來分流) 沒增加header user-agent 會直接503什麼都看不到
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
    }
    #透過request套件抓下url資料
    requ = requests.get(link, headers=header)
    #初步檢視抓到的資料結構
    rejs = requ.json()
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'createdAt':rejs['createdAt'],
          'commentCount':rejs['commentCount'],
          'forumName':rejs['forumName'],
          'gender':rejs['gender'],
          'likeCount':rejs['likeCount'],
          'topics':rejs['topics']}],
        #在 DataFrame 加一個欄位標籤標註為憂鬱症topic文章
        columns=['ID','title','content','createdAt','commentCount',
             'forumName','gender','likeCount','topics','depression']))

In [3]:
url = 'https://www.dcard.tw/_api/posts?popular=false&limit=100'
header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
}
resq = requests.get(url, headers=header)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Crawl(rejs[i]['id']), ignore_index=True)
print(df.shape)
df

(100, 10)


,ID,title,content,createdAt,commentCount,forumName,gender,likeCount,topics,depression
0,234113091,中華民國行憲後總統評分,現任不算數 別問我蔡英文做的怎樣了\n\n（以憲政後為準）\n\nA - 優秀，台灣囡仔\n...,2020-07-25T08:12:15.351Z,0,時事,M,0,"[總統, 歷史, 討論, 中華民國]",NaN
1,234113087,(問)牙齒矯正拔牙,我有蠻嚴重的暴牙\n在諮詢牙套相關事宜時，醫生有告知要上排要拔2顆牙齒，是拔兩側的犬齒\n已...,2020-07-25T08:11:32.447Z,1,牙齒矯正,F,0,[],NaN
2,234113086,送走皮耶羅之後（超主觀刷島民part2.2,oh nooooo 我跟男友又辛苦刷了74票\n也記錄下來要給你們看…但switch上傳問題...,2020-07-25T08:11:14.250Z,0,動物森友會,F,0,[],NaN
3,234113085,Switch 專題,拜託救救我們的專題😭\n動動你們的小手填一下\nhttps://docs.google.co...,2020-07-25T08:11:11.013Z,0,閒聊,M,0,"[專題, 求解]",NaN
4,234113081,全民網紅：創造被動收入的平台,全民網紅：是一個可以創造被動收入的平台。方法是到平台上完成每天的廣告任務，不同會員等級每天能...,2020-07-25T08:10:43.636Z,0,理財,F,1,"[被動收入, 投資, 理財, 分享]",NaN
...,...,...,...,...,...,...,...,...,...,...
95,234112900,有人有吃過台北三創的KIRABASE嗎？,"下禮拜想去台北, 順便要去逛光華三創, 想問看看有沒有人吃過這家\n這家看粉專好像是跟王領騎...",2020-07-25T07:28:34.020Z,1,美食,M,2,"[台北, 美食, Kirabase]",NaN
96,234112899,時旅幫大家刷藝術品,請先留言自己缺的藝術品\n上船買一件2票不管大小畫\n現貨一件5票\n贗品一件1票\n如果有...,2020-07-25T07:28:19.775Z,12,動物森友會,F,2,[動物森友會],NaN
97,234112898,到底何謂有穿沒搭,這是從高二開始看迪卡後就開始產生的疑問ಠ_ಠ\n所以我下定決心只要在得到迪卡帳號\n馬上發文...,2020-07-25T07:27:41.298Z,0,穿搭,F,2,"[穿搭, 問題, 日常穿搭]",NaN
98,234112897,燙捲後的整理,最近去燙頭髮，剛燙完非常滿意\n但回家整理後就不太捲，而且樣子也不太一樣\n我洗完頭是先分左...,2020-07-25T07:27:18.791Z,0,美髮,F,2,"[燙髮, 問題, 求解]",NaN


In [4]:
last = ''
error = []
for j in range(10):
    #找出爬出資料的最後一筆ID
    #last = str(int(df.tail(1).ID))
    url = 'https://www.dcard.tw/_api/posts?popular=false&limit=100' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        try:
            df = df.append(Crawl(rejs[i]['id']), ignore_index=True)
        except:
            error.append(rejs[i]['id'])
            continue
    last ='&before=' + str(int(df.tail(1).ID))
print(df.shape)
df['depression'].fillna(value=0, inplace=True)
df

(1100, 10)


,ID,title,content,createdAt,commentCount,forumName,gender,likeCount,topics,depression
0,234113091,中華民國行憲後總統評分,現任不算數 別問我蔡英文做的怎樣了\n\n（以憲政後為準）\n\nA - 優秀，台灣囡仔\n...,2020-07-25T08:12:15.351Z,0,時事,M,0,"[總統, 歷史, 討論, 中華民國]",0.0
1,234113087,(問)牙齒矯正拔牙,我有蠻嚴重的暴牙\n在諮詢牙套相關事宜時，醫生有告知要上排要拔2顆牙齒，是拔兩側的犬齒\n已...,2020-07-25T08:11:32.447Z,1,牙齒矯正,F,0,[],0.0
2,234113086,送走皮耶羅之後（超主觀刷島民part2.2,oh nooooo 我跟男友又辛苦刷了74票\n也記錄下來要給你們看…但switch上傳問題...,2020-07-25T08:11:14.250Z,0,動物森友會,F,0,[],0.0
3,234113085,Switch 專題,拜託救救我們的專題😭\n動動你們的小手填一下\nhttps://docs.google.co...,2020-07-25T08:11:11.013Z,0,閒聊,M,0,"[專題, 求解]",0.0
4,234113081,全民網紅：創造被動收入的平台,全民網紅：是一個可以創造被動收入的平台。方法是到平台上完成每天的廣告任務，不同會員等級每天能...,2020-07-25T08:10:43.636Z,0,理財,F,1,"[被動收入, 投資, 理財, 分享]",0.0
...,...,...,...,...,...,...,...,...,...,...
1095,234111063,如果,如果我們現在還在一起妳是不是還會每天都要我說對妳說我愛妳晚安呢每天都會跟我視訊\n每天我下班...,2020-07-24T18:52:58.875Z,1,彩虹,F,9,"[彩虹, 祝福]",0.0
1096,234111062,在職專班與一般生的差別,最近在職專班吵得沸沸揚揚\n\n雖然在職專班跟一般生的畢業證書通常會有差「在職專班」這幾個字...,2020-07-24T18:52:02.128Z,8,考試,M,3,"[在職專班, 大學, 研究所, 時事, 政策]",0.0
1097,234111061,YouTube 搜尋欄 lag解決辦法,不知道大家是否有遇到常常用電腦版YouTube 搜尋打字的時候（我自己是Chrome瀏覽器）...,2020-07-24T18:51:46.606Z,0,3C,M,0,"[電腦, Windows, Youtube]",0.0
1098,234111059,專櫃粉底液挑選（混乾/中性肌）,妹妹最近回家\n知道她開始有練習化妝\n她現在用的是 integrate的粉餅+media橘...,2020-07-24T18:50:03.927Z,17,美妝,F,10,"[專櫃, 粉底液]",0.0


In [7]:
#篩選憂鬱症topic文章
dep=df[pd.DataFrame(df.topics.tolist()).isin(['憂鬱症']).any(1)]
df.loc[pd.DataFrame(df.topics.tolist()).isin(['憂鬱症']).any(1),'depression']=1
dep

,ID,title,content,createdAt,commentCount,forumName,gender,likeCount,topics,depression
580,234112151,你說我聽，我說你聽,你說我聽\n曾經是種力量\n如今成為傷痛\n\n我說你聽\n當時候的秘密\n成為飯後話題\n...,2020-07-25T04:24:23.354Z,0,心情,F,1,"[焦慮症, 憂鬱症]",1.0
1075,234111106,Hi 我也是憂鬱症患者,Hi 我想用紀錄的方式說說這幾年的心路歷程.\n我是位憂鬱症患者，吃了一年半的憂鬱症藥物才...,2020-07-24T19:08:24.360Z,7,心情,F,12,"[心情, 心累了, 憂鬱症]",1.0
